In [1]:
import os
import time
from pathlib import Path
from dotenv import load_dotenv
from fastapi import FastAPI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from IPython.display import display, Markdown

from talking_robo.prompt.prompt_handler import PromptHandler

env_path = os.path.join(Path.cwd().parent, ".env")
load_dotenv(env_path)

/Users/aum/Desktop/Programming/llm/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
def colorize_text(text):
    for word, color in zip(["Instruction"], ["blue"]):
        text = text.replace(f"{word}:\n", f"**<font color='{color}'>{word}:</font>**\n\n")
    for word, color in zip(["Response"], ["green"]):
        text = text.replace(f"\n\n{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text
template = "Instruction:\n{instruction}\n\nResponse:\n"

### Try HuggingFaceHub

In [2]:
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint
llm_huggingface=HuggingFaceEndpoint(repo_id="google/gemma-1.1-2b-it",temperature=1,max_new_tokens=1000)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/aum/.cache/huggingface/token
Login successful


In [3]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(
    input_variables=['question'],
    template='For the following question, give the answer in the format defined: question: {question}')
    # PromptTemplate(input_variables=['country'],
# template="The highest mountain in {country} ?")

prompt_template

PromptTemplate(input_variables=['question'], template='For the following question, give the answer in the format defined: question: {question}')

In [4]:
from langchain.chains import LLMChain

# using hugging face open source LLM
chain=LLMChain(llm=llm_huggingface,prompt=prompt_template)
print(chain.invoke({"question": "What are decorators in Python?"}))

{'question': 'What are decorators in Python?', 'text': '\n\n**Answer:** Decorators in Python are a way to modify the behavior of functions or classes by adding additional functionality around them. They allow programmers to add new attributes, methods, or even modify the way existing attributes and methods are called.'}


### Pipeline run 1

In [3]:
import sys

mod_path = os.path.join(Path.cwd().parent)
print(mod_path)
if mod_path not in sys.path:
    sys.path.append(mod_path)

/Users/aum/Desktop/Programming/llm


In [4]:
def load_model():
    start_time = time.time()
    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it",
                                              # device_map="auto",
                                              token=os.environ.get("hugging_face_token")
                                              )
    model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it",
                                                 # device_map="auto",
                                                 token=os.environ.get("hugging_face_token"),
                                                 use_cache=True
                                                 # quantization_config=quantization_config
                                                 )
    print(f"Model loaded in {time.time() - start_time}s.")
    return tokenizer, model

TOKENIZER, MODEL = load_model()

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.51s/it]


Model loaded in 13.167423963546753s.


In [5]:
def get_model_inference(
    input_question: str
):
    llm_pipeline = pipeline(
        "text-generation",
        model=MODEL, 
        tokenizer=TOKENIZER, 
        max_new_tokens=5000
    )
    huggingface_pipeline = HuggingFacePipeline(
        pipeline=llm_pipeline
    )
    # Prompt chain
    chain_object = PromptHandler(
        user_question=input_question,
        hugging_face_pipeline=huggingface_pipeline,
        model="gemini"
    )
    instr = chain_object.instructions
    chain = chain_object.generated_prompt
    print(f'HERE -> {chain}')
    response = chain_object.llm_output
    
    # chain = prompt | huggingface_pipeline
    # response = chain.invoke({'question': input_question, 'format_instructions': instr})
    
    # response = llm_pipeline(chain)
    # Invoking the llm
    # response = chain.run(
    #     {"question": input_question},
    #     {"format_instructions": chain_object.generated_instructions}
    # )

    return chain


In [6]:
ans = get_model_inference("How do we use decorators in Python?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [35]:
llm_pipeline = pipeline(
        "text-generation",
        model=MODEL, 
        tokenizer=TOKENIZER, 
        max_new_tokens=100
    )
huggingface_pipeline = HuggingFacePipeline(
    pipeline=llm_pipeline
)


In [36]:
a = huggingface_pipeline.generate(['I am happy today because'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [8]:
input_ids = TOKENIZER(ans, return_tensors="pt")#.to("mps")
inf_start_time = time.time()
outputs = MODEL.generate(**input_ids, max_length=1000)
print(f"Inference completed and output generated in {time.time() - inf_start_time}s")

Inference completed and output generated in 622.9946038722992s


In [15]:
display(Markdown(TOKENIZER.decode(outputs[0])))

<bos>For the following question, give the answer in the format defined:

question: How do we use decorators in Python?

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Explanation": string  // Provide an explanation to the main entity of the question. This entity can either be a coding paradigm or a coding concept.

	"Advantages": string  // List down the advantages or benefits of using the entity defined in the question.
	"Example": string  // Give an example of the usage of the main entity of the question. Make sure this example can be run on the machine and is error free.
}
```
**Answer:**

```json
{
	"Explanation": "Decorators allow you to modify the behavior of a function or class without changing its source code. They are defined using the `@` symbol followed by the name of the decorator function.",
	"Advantages": "They allow you to modify the behavior of a function or class without changing its source code. They are also useful for creating reusable code.",
	"Example": "```python
from functools import wraps

def log_execution(func):
    def wrapper(*args, **kwargs):
        print("Calling function:", func.__name__)
        result = func(*args, **kwargs)
        print("Function:", func.__name__, "completed.")
        return result
    return wrapper
```

This code defines a decorator called `log_execution` that takes a function as input and wraps it. When the decorated function is called, it prints a message before and after executing the function and also returns the result.<eos>

### Pipeline Run #2

In [3]:
def load_model():
    llm=HuggingFaceEndpoint(
        repo_id="google/gemma-1.1-2b-it",
        temperature=1,
        max_new_tokens=1000
    )

    return llm

LLM_HUGGINGFACE = load_model()

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/aum/.cache/huggingface/token
Login successful


In [4]:
def get_model_inference(
    input_question: str
):
    # Prompt chain
    chain_object = PromptHandler(
        user_question=input_question,
        hugging_face_pipeline=LLM_HUGGINGFACE,
        model="gemini"
    )
    response = chain_object.response
    output_parser = chain_object.llm_output_parser

    return response, output_parser

In [8]:
output, parser = get_model_inference("What are decorators in Python?")

In [13]:
output_dict = parser.parse(output['text'])

In [17]:
output_dict.keys()

dict_keys(['Explanation', 'Advantages', 'Example'])

In [16]:
output_dict['Explanation']

'Decorators are lightweight function objects that add additional functionality to a function at runtime.'

In [18]:
output_dict['Advantages']

'\n        * **Enhance existing functionality:** Allow functions to be easily extended or modified.\n        * **Organize code:** Group related functionality into separate decorators.\n        * **Improve readability:** Make code more concise and readable by adding context-specific decorators.\n    '

In [19]:
output_dict['Example']

'```python\nfrom functools import wraps\nfrom time import sleep\n\n@wraps(sleep)\ndef slow_sleep(seconds):\n    '

In [22]:
display(Markdown(output['text']))

```
{
    "Explanation": "Decorators are lightweight function objects that add additional functionality to a function at runtime.",

    "Advantages": "
        * **Enhance existing functionality:** Allow functions to be easily extended or modified.
        * **Organize code:** Group related functionality into separate decorators.
        * **Improve readability:** Make code more concise and readable by adding context-specific decorators.
    ",
    "Example": "```python
from functools import wraps
from time import sleep

@wraps(sleep)
def slow_sleep(seconds):
    """
    Decorator that slows down the execution of a function by a specified number of seconds.

    Args:
        seconds: The number of seconds to slow down the execution by.
    """
    def wrapper(*args, **kwargs):
        print(f'Sleeping for {seconds} seconds...')
        result = sleep(seconds)
        print(f'Function completed in {result.real} seconds.')
        return result
    return wrapper
```

**Answer:**

```json
{
    "Explanation": "Decorators are lightweight function objects that add additional functionality to a function at runtime.",

    "Advantages": "
        * Enhance existing functionality
        * Organize code
        * Improve readability
    ",
    "Example": "```python
from functools import wraps
from time import sleep

@wraps(sleep)
def slow_sleep(seconds):
    """
    Decorator that slows down the execution of a function by a specified number of seconds.

    Args:
        seconds: The number of seconds to slow down the execution by.
    """
    def wrapper(*args, **kwargs):
        print(f'Sleeping for {seconds} seconds...')
        result = sleep(seconds)
        print(f'Function completed in {result.real} seconds.')
        return result
    return wrapper
```